In [38]:
seed = 2023
from nlpsig_networks.scripts.seqsignet_functions import obtain_SeqSigNet_input
from nlpsig_networks.scripts.swnu_network_functions import obtain_SWNUNetwork_input
from nlpsig_networks.utils import obtain_paths_mask, obtain_signatures_mask

In [2]:
import os

os.chdir("Rumours/")

## Rumours

In [3]:
%run load_sbert-embeddings.py

In [4]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [5]:
len(df_rumours)

5568

In [6]:
shift = 3
window_size = 5
n = 11

In [96]:
x_data = obtain_SWNUNetwork_input(
    method="umap",
    dimension=3,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    k=35,
    features=["time_encoding", "timeline_index"],
    standardise_method=None,
    add_features_in_path=False,
    path_indices=[0, 1, 2, 3],
)

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


In [97]:
x_data["x_data"]["path"] = torch.stack(
    [
        x_data["x_data"]["path"][:, (i * shift) : (i * shift + window_size), :]
        for i in range(n)
    ],
    dim=1,
)

In [75]:
x_data.keys()

dict_keys(['x_data', 'input_channels', 'embedding_dim', 'num_features'])

In [98]:
x_data["x_data"]

{'path': tensor([[[[-7.2296,  2.7356,  8.0972],
           [ 0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000]],
 
          [[ 0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000]],
 
          [[ 0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000]],
 
          [[ 0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000]],
 
          [[ 0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000],
           [ 0.0000,

In [99]:
x_data["x_data"]["path"].shape

torch.Size([4, 11, 5, 3])

In [100]:
x_data["x_data"]["features"]

tensor([[ 6.4823e-02,  9.3748e-03, -3.3364e-02,  ...,  1.8542e-03,
          2.0148e+03,  1.0000e+00],
        [-2.9260e-02, -5.3905e-02, -3.9863e-02,  ...,  9.3606e-02,
          2.0148e+03,  2.0000e+00],
        [-3.1863e-02,  5.5065e-02, -2.4028e-02,  ...,  9.3310e-02,
          2.0148e+03,  3.0000e+00],
        [ 5.3822e-02,  9.4921e-03, -9.6542e-03,  ...,  1.1833e-02,
          2.0148e+03,  4.0000e+00]])

In [101]:
x_data["x_data"]["features"].shape

torch.Size([4, 386])

In [80]:
x_data["input_channels"]

3

In [81]:
x_data["embedding_dim"]

384

In [82]:
x_data["num_features"]

2

In [32]:
x_data["x_data"]["path"][0, 0, :, :]

tensor([[-7.2296,  2.7356,  8.0972],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)

In [83]:
path = x_data["x_data"]["path"]

In [85]:
path[0][0]

tensor([[-7.2296,  2.7356,  8.0972],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)

In [95]:
torch.equal(torch.sum(torch.sum(path, (2, 3)) != 0, 1), seq_lengths_prev)

True

In [50]:
seq_lengths_prev = torch.sum(torch.sum(torch.sum(path, 1) != 0, 1) != 0, 1)

In [71]:
torch.sum(torch.sum(torch.sum(path, 1) != 0, 1) != 0, 1)

tensor([1, 1, 1,  ..., 8, 8, 8])

In [69]:
seq_lengths_prev

tensor([1, 1, 1,  ..., 8, 8, 8])

In [51]:
seq_lengths_prev.shape

torch.Size([5568])

In [52]:
seq_lengths_prev[0:10]

tensor([1, 1, 1, 2, 2, 1, 1, 1, 2, 1])

In [105]:
path[0][0]

tensor([[-7.2296,  2.7356,  8.0972],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)

In [106]:
path[0][1]

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], dtype=torch.float64)

In [114]:
path_flat = path.view(-1, path.shape[2], path.shape[3])

In [115]:
torch.equal(path_flat, path_flattened)

True

In [107]:
path_flattened = path.flatten(0, 1)
path_flattened.shape

torch.Size([61248, 5, 3])

In [117]:
path_unflattened = path_flattened.unflatten(0, (path.shape[0], path.shape[1]))

In [118]:
torch.equal(path, path_unflattened)

True